In [15]:
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
# 구글 드라이브 마운트
drive.mount('/content/drive')
def loadfile(path):
    X = []
    Y = []
    for label in ('0', '25', '50', '75', '100'):
        print("Loading text files for the label: " + label)
        label_path = os.path.join(path, label)
        for filename in os.listdir(label_path):
            if filename.endswith('.txt'):
                with open(os.path.join(label_path, filename), 'r', encoding='utf-8') as file:
                    text = file.read()
                # 레이블을 숫자로 변환
                if label == '0':
                    Y.append(0)
                elif label == '25':
                    Y.append(1)
                elif label == '50':
                    Y.append(2)
                elif label == '75':
                    Y.append(3)
                elif label == '100':
                    Y.append(4)
                # 텍스트 데이터를 X에 추가
                X.append(text)
    X = np.array(X)
    Y = np.array(Y)
    return X, Y
# 경로를 구글 드라이브 경로로 변경
directory_path = '/content/drive/MyDrive/likeability_Son/'
# loadfile 함수 호출
X, Y = loadfile(directory_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading text files for the label: 0
Loading text files for the label: 25
Loading text files for the label: 50
Loading text files for the label: 75
Loading text files for the label: 100


In [18]:
print("X shape:", X.shape)
print("Y shape:", Y.shape)

AttributeError: ignored

In [21]:
#4-1. 이모지 사용 함수
def count_emojis(text):
    emoji_pattern = re.compile('['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        ']', flags=re.UNICODE)
    return len(emoji_pattern.findall(text))


#4.2. 대화 양방향성 관련 함수
def check_bidirectional_conversation(text):
    a_contributions = len(re.findall(r'A:', text))
    b_contributions = len(re.findall(r'B:', text))
    return a_contributions > 0 and b_contributions > 0


# 4-3. 답장 속도 기반 호감도 예측
from datetime import datetime
import re
import numpy as np
# 대화 데이터를 .txt 파일에서 불러오기
conversation = []
for label in ('0', '25', '50', '75', '100'):
    label_path = os.path.join(directory_path, label)
    for filename in os.listdir(label_path):
        if filename.endswith('.txt'):
            with open(os.path.join(label_path, filename), 'r', encoding='utf-8') as file:
                conversation.extend(file.readlines())


# 호감도 레이블 생성 함수
def create_likeability_labels(conversation):
    response_times = []
    last_message_time = None
    for line in conversation:
        if line.startswith("A:") or line.startswith("B:"):
            # 시간 정보 추출 및 변환
            time_str = re.search(r'\((\d{2}):(\d{2})\)', line)
            if time_str:
                hours, minutes = map(int, time_str.groups())
                current_time = hours * 60 + minutes  # Convert to minutes
                # 답장 속도 계산
                if last_message_time is not None:
                    response_time = current_time - last_message_time
                    response_times.append(response_time)
                last_message_time = current_time
    # 평균 응답 시간 계산 및 호감도 레이블 생성
    avg_response_times = np.mean(response_times) if response_times else 0
    likeability_labels = []
    for response_time in response_times:
        if response_time <= 10:
            likeability = 100
        elif response_time <= 30:
            likeability = 75
        elif response_time <= 60:
            likeability = 50
        elif response_time <= 180:
            likeability = 25
        else:
            likeability = 0
        likeability_labels.append(likeability)
    return likeability_labels
# 대화 데이터를 기반으로 호감도 레이블 생성
likeability_labels = create_likeability_labels(conversation)

In [22]:
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 데이터 전처리
# 1) 특수문자 제거
X_cleaned = [re.sub(r"[^가-힣A-Za-z0-9]", " ", text) for text in X]

# 2) & 3) 토크나이징
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_cleaned)
X_seq = tokenizer.texts_to_sequences(X_cleaned)

# 4) & 5) 포스트패딩
max_sequence_length = 100
X_padded = pad_sequences(X_seq, maxlen=max_sequence_length)

# 6) 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_padded, Y, test_size=0.2, random_state=42)


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

# 모델 정의
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model.add(GRU(128, return_sequences=True))
model.add(GRU(128))
model.add(Dense(5, activation='softmax'))  # 클래스가 5개

# 모델 구조 출력
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          1280000   
                                                                 
 gru_1 (GRU)                 (None, 100, 128)          99072     
                                                                 
 gru_2 (GRU)                 (None, 128)               99072     
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 1478789 (5.64 MB)
Trainable params: 1478789 (5.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20
10/10 [==============================] - 10s 194ms/step - loss: 1.5830 - accuracy: 0.3125 - val_loss: 1.5489 - val_accuracy: 0.4125
Epoch 2/20
10/10 [==============================] - 1s 117ms/step - loss: 1.3384 - accuracy: 0.6031 - val_loss: 1.2308 - val_accuracy: 0.4750
Epoch 3/20
10/10 [==============================] - 1s 123ms/step - loss: 0.7080 - accuracy: 0.6906 - val_loss: 1.0039 - val_accuracy: 0.5625
Epoch 4/20
10/10 [==============================] - 1s 78ms/step - loss: 0.3008 - accuracy: 0.9344 - val_loss: 1.0040 - val_accuracy: 0.6500
Epoch 5/20
10/10 [==============================] - 1s 84ms/step - loss: 0.1135 - accuracy: 0.9719 - val_loss: 1.1901 - val_accuracy: 0.6375
Epoch 6/20
10/10 [==============================] - 1s 83ms/step - loss: 0.0989 - accuracy: 0.9688 - val_loss: 1.0629 - val_accuracy: 0.7125
Epoch 7/20
10/10 [==============================] - 1s 73ms/step - loss: 0.0510 - accuracy: 0.9812 - val_loss: 1.2130 - val_accuracy: 0.7375
Epoch 8/2

In [26]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

4/4 [==============================] - 0s 9ms/step - loss: 0.7629 - accuracy: 0.8200
Test Loss: 0.7629058957099915, Test Accuracy: 0.8199999928474426
